In [1]:
import os
# pyspark --packages graphframes:graphframes:0.3.0-spark2.0-s_2.11 --jars graphframes-0.3.0-spark2.0-s_2.11.jar
pyspark_submit_args = '--packages graphframes:graphframes-0.8.0-spark3.0-s_2.12 pyspark-shell'# --jars graphframes-0.8.0-spark3.0-s_2.12.jar'
pyspark_submit_args = '--jars graphframes-0.8.0-spark3.0-s_2.12.jar pyspark-shell'

os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args
import findspark
findspark.init()
findspark.find()
import pyspark
conf = pyspark.SparkConf()
sc = pyspark.SparkContext(conf=conf)

# from pyspark import *


In [2]:
from pyspark import *
from pyspark.sql import *
from graphframes import *
sparkSession = SparkSession.builder.appName('A4').master('local').getOrCreate()
# spark = SparkSession.builder.appName('BDA_A2').master('local').getOrCreate()

In [3]:
dataset = open('soc-Epinions1.txt', 'r')
lines = dataset.readlines()
lines = lines[4:] # skipping the comments in file

In [4]:
vertices = set()
edges = []
for line in lines:
    source, dest = line.split()
    vertices.add((source,))
    vertices.add((dest,))
    edges.append((source, dest))
#     print(source, dest)
#     break
vertices = list(vertices)

In [5]:
vertices_df = sparkSession.createDataFrame(vertices, ["id"])
edges_df = sparkSession.createDataFrame(edges, ["src", "dst"])

In [6]:
graph = GraphFrame(vertices_df, edges_df)

In [8]:
graph.inDegrees.show()
alpha = 0.8

+----+--------+
|  id|inDegree|
+----+--------+
|1572|     180|
|2069|      51|
| 296|     121|
|1512|     131|
|2904|      53|
|3414|      16|
| 829|      63|
| 675|      33|
|2088|      80|
|2162|      61|
|4032|      17|
|2294|      56|
|6613|      59|
|6731|       8|
|1436|     181|
| 691|      47|
|9009|      34|
|3210|      34|
|9586|       1|
|9993|       2|
+----+--------+
only showing top 20 rows



In [11]:
results = graph.pageRank(resetProbability=1-alpha, maxIter=20)
results.vertices.sort('pagerank' , ascending=False).show(5)

+----+------------------+
|  id|          pagerank|
+----+------------------+
|  18| 325.8853046489408|
| 737|212.66576555045725|
|1719|147.22687135149997|
| 118| 145.0670191471319|
| 790|142.33446484041733|
+----+------------------+
only showing top 5 rows



In [12]:
# https://graphframes.github.io/graphframes/docs/_site/api/python/graphframes.html#graphframes.GraphFrame.parallelPersonalizedPageRank
results2 = graph.pageRank(resetProbability=1-alpha, maxIter=20, sourceId="18")


In [13]:
results2.vertices.sort('pagerank' , ascending=False).show(5)

+----+--------------------+
|  id|            pagerank|
+----+--------------------+
|  18|  0.2140906165799031|
| 118|0.006807206592250927|
| 790|0.006075429899075041|
| 136|0.005978881253253703|
|1191|0.005723130330628861|
+----+--------------------+
only showing top 5 rows

